# Deploying Machine Learning Models using kubectl
This demo shows how you can interact directly with kubernetes using kubectl to create and manage runtime machine learning models.
<img src="images/deploy-graph.png" alt="predictor with canary" title="ml graph"/>

In [3]:
!kubectl label nodes apiserver-crd role=locust --overwrite

node "apiserver-crd" not labeled


## Start seldon-core

In [2]:
!helm install ../../helm-charts/seldon-core --name seldon-core --set grafana_prom_admin_password=password --set persistence.enabled=false --set cluster_manager.image.tag=0.2.16_kubeapi --set apife.image.tag=0.0.7_kubeapi

NAME:   seldon-core
LAST DEPLOYED: Sun Nov  5 14:43:04 2017
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/Service
NAME                      TYPE       CLUSTER-IP  EXTERNAL-IP  PORT(S)                     AGE
alertmanager              ClusterIP  10.0.0.75   <none>       80/TCP                      2s
seldon-apiserver          NodePort   10.0.0.152  <none>       8080:30032/TCP              2s
seldon-cluster-manager    NodePort   10.0.0.120  <none>       8080:30030/TCP              2s
grafana-prom              NodePort   10.0.0.103  <none>       80:30034/TCP                2s
kafka                     NodePort   10.0.0.185  <none>       9092:30010/TCP              2s
prometheus-node-exporter  ClusterIP  None        <none>       9100/TCP                    2s
prometheus-seldon         ClusterIP  10.0.0.45   <none>       80/TCP                      2s
redis                     ClusterIP  10.0.0.69   <none>       6379/TCP                    2s
zookeeper-1               ClusterIP  10

# !kubectl get deployments

# Integrating with Kubernetes API

In [4]:
!cat resources/model.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha1",
    "kind": "MLDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "mldeployment-example"
    },
    "spec": {
        "annotations": {
            "project_name": "FX Market Prediction"
        },
        "cluster": {},
        "id": "1",
        "name": "test deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictor": {
            "annotations": {
                "project_name": "FX Market Prediction"
            },
            "enabled": true,
            "id": "0",
            "imagePullSecrets": [],
            "name": "FX Market Predictor",
            "predictiveUnits": [
                {
                    "children": [],
                    "cluster_resources": {
                        "cpu": "",
                        "gpu": "",
                        "id": "2",
                        "image": "seldonio/mean_cla

## Create ML Deployment

In [5]:
!kubectl apply -f resources/model.json

mldeployment "mldeployment-example" created


In [6]:
!kubectl get mldeployments

NAME                   AGE
mldeployment-example   1s


In [7]:
!kubectl describe mldeployments mldeployment-example 

Name:         mldeployment-example
Namespace:    default
Labels:       app=seldon
Annotations:  kubectl.kubernetes.io/last-applied-configuration={"apiVersion":"machinelearning.seldon.io/v1alpha1","kind":"MLDeployment","metadata":{"name":"mldeployment-example","namespace":"default","selfLink":"/a...
API Version:  machinelearning.seldon.io/v1alpha1
Kind:         MLDeployment
Metadata:
  Cluster Name:                   
  Creation Timestamp:             2017-11-05T14:43:38Z
  Deletion Grace Period Seconds:  0
  Finalizers:
  Generate Name:  
  Generation:     0
  Initializers:   <nil>
  Owner References:
  Resource Version:  21594
  Self Link:         /apis/machinelearning.seldon.io/v1alpha1/namespaces/default/mldeployments/mldeployment-example
  UID:               b585fc4f-c237-11e7-825b-08002703d5aa
Spec:
  Annotations:
    Project _ Name:  FX Market Prediction
  Cluster:
  Id:              1
  Name:            test deployment
  Oauth _ Key:     oauth-key
  Oauth _ Secret:  oauth-secret

In [9]:
!kubectl get mldeployments mldeployment-example -o jsonpath='{.status}'

map[canaryReplicasReady:0 predictorReplicasReady:1]

## Get predictions

In [11]:
%%bash
API_SERVER=35.195.94.148:8080
MINIKUBE_SERVER=192.168.99.100:30032
SERVER=$API_SERVER
SERVER=$MINIKUBE_SERVER
#SERVER=0.0.0.0:8080
echo "$SERVER"
TOKEN=`curl -s -H "Accept: application/json" oauth-key:oauth-secret@${SERVER}/oauth/token -d grant_type=client_credentials | jq -r '.access_token'`
curl -s -H "Content-Type:application/json" -H "Accept: application/json" -H "Authorization: Bearer $TOKEN" \
    ${SERVER}/api/v0.1/predictions \
    -d '{"request":{"features":["a"],"ndarray":[[1.0],[2.0]]}}'

192.168.99.100:30032
{
  "meta": {
    "puid": "75cv36l67bc3ffem9k3f35rot4",
    "routing": {
    }
  },
  "response": {
    "features": ["proba"],
    "ndarray": [[0.12823373759251927], [0.2856375705089441]]
  }
}

## Update deployment with canary

In [12]:
!cat resources/model_with_canary.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha1",
    "kind": "MLDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "mldeployment-example"
    },
    "spec": {
        "annotations": {
            "project_name": "FX Market Prediction"
        },
        "cluster": {},
        "id": "1",
        "name": "test deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictor": {
            "annotations": {
                "project_name": "FX Market Prediction"
            },
            "enabled": true,
            "id": "0",
            "imagePullSecrets": [],
            "name": "FX Market Predictor",
            "predictiveUnits": [
                {
                    "children": [],
                    "cluster_resources": {
                        "cpu": "",
                        "gpu": "",
                        "id": "2",
                        "image": "seldonio/mean_cla

In [13]:
!kubectl apply -f resources/model_with_canary.json

mldeployment "mldeployment-example" configured


In [18]:
!kubectl get mldeployments mldeployment-example -o jsonpath='{.status}'

map[canaryReplicasReady:1 predictorReplicasReady:1]

In [19]:
%%bash
API_SERVER=35.189.249.200:8080
MINIKUBE_SERVER=192.168.99.100:30032
SERVER=$API_SERVER
SERVER=$MINIKUBE_SERVER
TOKEN=`curl -s -H "Accept: application/json" oauth-key:oauth-secret@${SERVER}/oauth/token -d grant_type=client_credentials | jq -r '.access_token'`
curl -s -H "Content-Type:application/json" -H "Accept: application/json" -H "Authorization: Bearer $TOKEN" \
    ${SERVER}/api/v0.1/predictions \
    -d '{"request":{"features":["a"],"ndarray":[[1.0],[2.0]]}}'

{
  "meta": {
    "puid": "ckhafbhvcci7eumr5ji8q0bphk",
    "routing": {
    }
  },
  "response": {
    "features": ["proba"],
    "ndarray": [[0.12823373759251927], [0.2856375705089441]]
  }
}

## Load test

In [20]:
!helm install --set oauth.key=oauth-key --set oauth.secret=oauth-secret --name loadtest ../../helm-charts/seldon-core-loadtesting

NAME:   loadtest
LAST DEPLOYED: Sun Nov  5 14:49:42 2017
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/ReplicationController
NAME             DESIRED  CURRENT  READY  AGE
locust-slave-1   1        1        0      0s
locust-master-1  1        1        0      0s

==> v1/Service
NAME             TYPE      CLUSTER-IP  EXTERNAL-IP  PORT(S)                                       AGE
locust-master-1  NodePort  10.0.0.26   <none>       5557:30376/TCP,5558:30483/TCP,8089:30169/TCP  0s

==> v1/Pod(related)
NAME                   READY  STATUS             RESTARTS  AGE
locust-slave-1-2sd7k   0/1    ContainerCreating  0         0s
locust-master-1-6gf8s  0/1    ContainerCreating  0         0s




# Tear down

In [21]:
!kubectl delete -f resources/model_with_canary.json

mldeployment "mldeployment-example" deleted


In [22]:
!helm delete loadtest --purge

release "loadtest" deleted


In [23]:
!helm delete seldon-core --purge

release "seldon-core" deleted
